Воспользуюсь несколькими библиотеками для упрощения работы.

In [1]:
!pip install surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-win_amd64.whl size=708757 sha256=6350213bcadaf69c71a21ed6d16e22f33df25f9f4dcc39e869046ba2a5706b9e
  Stored in directory: c:\users\kirpa\appdata\local\pip\cache\wheels\20\91\57\2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise


In [25]:
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import train_test_split, cross_validate

In [26]:
from scipy.sparse import csc_matrix
from scipy import sparse
import pandas as pd
import numpy as np

In [27]:
rates = pd.read_csv('rating.csv')

In [28]:
users_number = rates.userId.unique().shape[0]
movies_number = rates.movieId.unique().shape[0]
users_number, movies_number

(138493, 26744)

In [29]:
id_to_row = { dfid: row for row, dfid in enumerate(rates.userId.unique()) }
id_to_col = { dfid: col for col, dfid in enumerate(rates.movieId.unique()) }

In [30]:
prepared_data = rates.drop(columns='timestamp')
prepared_data['userId'] = np.array([id_to_row[dfid] for dfid in rates.userId])
prepared_data['movieId'] = np.array([id_to_col[dfid] for dfid in rates.movieId])
prepared_data.sample(n=10)

,userId,movieId,rating
17662939,122159,1571,4.0
15328433,105998,1644,3.0
9666916,66844,535,1.0
19065797,131903,12175,3.0
6239788,42908,2820,3.5
9179175,63443,6695,1.0
3550837,24211,4441,3.0
18222166,126014,1402,4.0
6642010,45721,610,0.5
15938910,110269,2165,1.5


In [31]:
user_film_matrix = csc_matrix((prepared_data['rating'].values, (prepared_data['userId'].values.astype(int), prepared_data['movieId'].values.astype(int))))

In [32]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(prepared_data.sample(n=10000)[['userId', 'movieId', 'rating']], reader)
type(data)

surprise.dataset.DatasetAutoFolds

In [33]:
trainset, testset = train_test_split(data, test_size=0.25)

svd_model = SVD()
svd_model.fit(trainset)

In [41]:
len(testset)

2500

In [44]:
predictions = svd_model.test(testset)
predictions[0:10]

[Prediction(uid=22411, iid=401, r_ui=4.0, est=3.7054813149850534, details={'was_impossible': False}),
 Prediction(uid=106756, iid=8098, r_ui=4.5, est=3.5324, details={'was_impossible': False}),
 Prediction(uid=128346, iid=1529, r_ui=3.5, est=3.6039194994355013, details={'was_impossible': False}),
 Prediction(uid=110410, iid=3448, r_ui=4.0, est=3.490923926723941, details={'was_impossible': False}),
 Prediction(uid=123326, iid=1264, r_ui=4.0, est=3.4791391739515287, details={'was_impossible': False}),
 Prediction(uid=103015, iid=858, r_ui=4.0, est=3.589746451897423, details={'was_impossible': False}),
 Prediction(uid=68290, iid=9, r_ui=5.0, est=4.197507729917382, details={'was_impossible': False}),
 Prediction(uid=90529, iid=4, r_ui=5.0, est=4.131749466903174, details={'was_impossible': False}),
 Prediction(uid=24150, iid=3313, r_ui=4.0, est=3.5324, details={'was_impossible': False}),
 Prediction(uid=127900, iid=784, r_ui=4.0, est=3.097374794209381, details={'was_impossible': False})]

In [55]:
accuracy.rmse(predictions)

RMSE: 1.0355


1.0355419243918176

Использую еще одну библиотеку для упрощения работы.

In [1]:
!pip install lightfm

  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-win_amd64.whl size=430786 sha256=7d70e6f2983c7c6c88e9f103f58d716664e3de33d2cc60b470ce475f8266e1aa
  Stored in directory: c:\users\kirpa\appdata\local\pip\cache\wheels\ec\bb\51\9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built lightfm


In [17]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

data = fetch_movielens(min_rating=3.0)

model = LightFM(loss='warp')
model.fit(data['train'], epochs=30, num_threads=2)

test_precision = precision_at_k(model, data['test'], k=5)

In [14]:
repr(data['train']), repr(data['test'])

("<943x1682 sparse matrix of type '<class 'numpy.float32'>'\n\twith 74627 stored elements in COOrdinate format>",
 "<943x1682 sparse matrix of type '<class 'numpy.int32'>'\n\twith 7893 stored elements in COOrdinate format>")

In [23]:
np.median(test_precision), test_precision.max(), test_precision.mean()

(0.0, 0.8, 0.10540828)

Много предсказаний, которые никак не попали в то, что нравится пользователю. Однако же есть и хорошие выдачи, 80% фильмов в них годятся для рекомендации.

In [ ]:
model = LightFM(loss='warp', learning_rate=0.90)
model = model.fit(user_film_matrix,
                  epochs=100, verbose=False)

In [ ]:
scores = pd.Series(model.predict(user, np.arange(n_items), item_features=user_film_matrix))